In [ ]:
import numpy as np
from env_1 import Env1
#from algos import *
import math
from scipy.stats import norm

In [ ]:
start_position = np.array([0.2, -0.2, 0.1])
goal_position = np.array([0.8, 0.15, 0.5])
xi0 = np.linspace(start_position, goal_position, 4)


#worst = np.load('results/worst_traj_B5.1.1.npy')
#best = np.load('results/best_traj_B5.1.1.npy')


# initialize environment
env = Env1(visualize=True)
#print(env.feature_count(worst, [255, 0, 0]))
#print(env.feature_count(best, [0, 255, 0]))
print(env.feature_count(xi0))

In [1]:
import pybullet as p
import pybullet_data
import time
import numpy as np

# Connect to PyBullet in GUI mode
p.connect(p.GUI)

# Set the path for PyBullet's data repository
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# Load the plane and Panda robot (fixed base)
p.loadURDF("plane.urdf")
robot_id = p.loadURDF("franka_panda/panda.urdf", useFixedBase=True)

# Load an object to pick (e.g., a box)
box_id = p.loadURDF("cube_small.urdf", basePosition=[0.6, 0, 0.02])

# Set gravity and simulation settings
p.setGravity(0, 0, -9.81)

# Get the end-effector link index (in Panda, it's link 11)
end_effector_index = 11

# Predefined gripper joint indices (Panda gripper uses joint 9 and 10)
left_finger_joint_index = 9
right_finger_joint_index = 10

# Define positions
object_position = [0.6, 0, 0.02]  # Position of the object (on the ground)
target_position = [0.4, 0.2, 0.02]  # Target position to place the object

# Function to move the arm to a desired position using inverse kinematics
def move_arm_to_position(target_pos, target_orn):
    joint_positions = p.calculateInverseKinematics(robot_id, end_effector_index, target_pos, target_orn)
    for i in range(7):  # The first 7 joints control the arm (Panda robot has 7 DoF)
        p.setJointMotorControl2(robot_id, i, p.POSITION_CONTROL, joint_positions[i])

# Open gripper function
def open_gripper():
    p.setJointMotorControl2(robot_id, left_finger_joint_index, p.POSITION_CONTROL, targetPosition=0.04)
    p.setJointMotorControl2(robot_id, right_finger_joint_index, p.POSITION_CONTROL, targetPosition=0.04)

# Close gripper function
def close_gripper():
    p.setJointMotorControl2(robot_id, left_finger_joint_index, p.POSITION_CONTROL, targetPosition=0.02)
    p.setJointMotorControl2(robot_id, right_finger_joint_index, p.POSITION_CONTROL, targetPosition=0.02)

# Grasp object function to simulate attaching it to the robot gripper
def grasp_object(obj_id):
    # Attach the object to the end-effector so it moves with the gripper
    constraint_id = p.createConstraint(robot_id, end_effector_index, obj_id, -1, p.JOINT_FIXED,
                                       [0, 0, 0], [0, 0, 0], [0, 0, 0])
    return constraint_id

# Release object function to detach it from the gripper
def release_object(constraint_id):
    p.removeConstraint(constraint_id)

# Get the initial orientation of the end-effector
_, _, _, _, _, ee_orn = p.getLinkState(robot_id, end_effector_index)

# Main simulation loop
for _ in range(500):
    p.stepSimulation()
    time.sleep(1./240.)

# Step 1: Move the arm above the object (using inverse kinematics)
hover_above_object = [object_position[0], object_position[1], object_position[2] + 0.2]
move_arm_to_position(hover_above_object, ee_orn)

for _ in range(1000):
    p.stepSimulation()
    time.sleep(1./240.)

# Step 2: Lower the arm to grasp the object
move_arm_to_position(object_position, ee_orn)

for _ in range(1000):
    p.stepSimulation()
    time.sleep(1./240.)

# Step 3: Close the gripper to grasp the object
close_gripper()

for _ in range(1000):
    p.stepSimulation()
    time.sleep(1./240.)

# Step 4: Attach the object to the gripper
constraint_id = grasp_object(box_id)

# Step 5: Lift the object up
lift_position = [object_position[0], object_position[1], object_position[2] + 0.2]
move_arm_to_position(lift_position, ee_orn)

for _ in range(1000):
    p.stepSimulation()
    time.sleep(1./240.)

# Step 6: Move the object to the target position
target_position_above = [target_position[0], target_position[1], target_position[2] + 0.2]
move_arm_to_position(target_position_above, ee_orn)

for _ in range(1000):
    p.stepSimulation()
    time.sleep(1./240.)

# Step 7: Lower the object to the target position
move_arm_to_position(target_position, ee_orn)

for _ in range(1000):
    p.stepSimulation()
    time.sleep(1./240.)

# Step 8: Open the gripper to release the object
open_gripper()

for _ in range(1000):
    p.stepSimulation()
    time.sleep(1./240.)

# Step 9: Release the object from the gripper
release_object(constraint_id)

# Step 10: Move the arm back to a neutral position (e.g., hover position)
neutral_position = [0.5, 0, 0.5]  # A position where the robot's arm hovers
move_arm_to_position(neutral_position, ee_orn)

for _ in range(1000):
    p.stepSimulation()
    time.sleep(1./240.)

# Disconnect the simulation
p.disconnect()


pybullet build time: Nov 28 2023 23:52:03


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 4090/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 560.35.03
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 560.35.03
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 4090/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
ven = NVIDIA Corporation
numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
dest